# Lab6

---

## Task

Реализовать метод Якоби поиска всех собственных чисел. Использовать две какие-либо стратегии выбора обнуляемого элемента.

- Вычисления проводить до достижения точности ε.
- Варьируя ε, скажем от 10<sup>-2</sup> до 10<sup>-5</sup>, изучить зависимость количества итераций от ε.
- Обязательно протестировать на матрице Гильберта порядка > 5
- Выводить количество итераций.
- По теореме Гершгорина определить область, в которую должны попадать с.ч. матрицы. Проверить, действительно ли найденные значения в область попали.

# Solution

---

In [ ]:
import math
import numpy as np
from scipy import linalg as la

from utils.matrices import *

In [ ]:
def sum_row_no_diag(A):
    """
    Sum of all row elements without diagonal one
    """
    n, _ = A.shape
    return sum([A[i][j] for i in range(n) for j in range(n) if i != j])

In [ ]:
def jacobi_max_module(A, error):
    """
    Using max module element
    """
    n, _ = A.shape

    max_el = 0
    for i in range(n - 1):
        for j in range(i + 1, n):
            if abs(A[i, j]) >= max_el:
              max_el = abs(A[i, j])
              k = i
              l = j 

    vects = matrix_rotate(A, k, l)
    R = matrix_rotate(A, k, l)
    A = (R.T @ A) @ R

    iter = 0
    while max_el > error:
        
        R = matrix_rotate(A, k, l)
        A = (R.T @ A) @ R
        vects = vects @ R
        max_el = 0

        for i in range(n - 1):
            for j in range(i + 1, n):
                if abs(A[i,j]) >= max_el:
                    max_el = abs(A[i,j])
                    k = i
                    l = j

        iter += 1
    
    eigs = np.diagonal(A)
    
    return eigs, vects, iter

In [ ]:
def matrix_rotate(A, k, l):
    x   = 2 * A[k][l] / (A[k][k] - A[l][l])
    cos = math.cos(1 / 2 * math.atan(x))
    sin = math.sin(1 / 2 * math.atan(x))
    
    R = np.eye(len(A))
    R[k][k] = cos
    R[k][l] = -sin
    R[l][k] = sin
    R[l][l] = cos
    
    return R

In [ ]:
def jacobi_cyclic_selection(A_, error):
    """
    Using cyclic selection strategy. Enumerate diagonal elements and reset round
    """
    n, _ = A_.shape
    A = A_.copy()
    
    sum = 0
    k = 0
    l = 1
    
    vects = matrix_rotate(A, k, l)
    R = matrix_rotate(A, k, l)
    A = (R.T @ A) @ R
    sum = sum_row_no_diag(A)
    k = 0
    l = 2

    iter = 0
    while sum > error:
        R = matrix_rotate(A, k, l)
        A = (R.T @ A) @ R
        vects = vects @ R
        
        if l < n - 1 and l + 1 != k:
            l += 1
        elif l < n - 2 and l + 1 == k:
            l += 2
        elif l < n - 1 and l + 1 == k and l + 1 == n - 1 and k < n - 1:
            k += 1
            l = 0
        elif k < n - 1:
            k +=1
            l = 0
        else:
            k = 0
            l = 1    
        
        sum = sum_row_no_diag(A)
        iter += 1

    eigs = np.diagonal(A)
    
    return eigs, vects, iter

In [ ]:
def check_eigs_range(A):
    """
    Using Gershgorin's theorem get the possible range of eigenvalues
    """
    left = 10000
    right = - left

    for row in A: 
        centre = row[0] 
        radius = np.sum(np.absolute(row)) - np.absolute(row[0])
        
        if centre - radius < left:
            left = centre - radius
        
        if centre + radius > right:
            right = centre + radius
        
    
    return left, right

## Experimental research

In [ ]:
def experiment(A):
     for error_deg in range(-2, -5, -1):
          error = 10**(error_deg)
          print(f"=================== Max absolute eigenvalue. Error = {error} ===================")
          
          actual_max_eigs, actual_max_vects, iter = jacobi_max_module(A, error)
          print(f"Max module method:")
          print(f"      eigenvalues   : {actual_max_eigs}")
          print(f"      eigenvectors  :\n{actual_max_vects}")
          print(f"Iterations: {iter}\n")
          
          actual_cyc_eigs, actual_cyc_vects, iter = jacobi_cyclic_selection(A, error)
          print(f"Cyclic selection method:")
          print(f"      eigenvalues   : {actual_cyc_eigs}")
          print(f"      eigenvectors  :\n{actual_cyc_vects}")
          print(f"Iterations: {iter}\n")
          
     
     expected_eigs, expected_vects = la.eig(A)
     print("=============================================================================")
     print(f"Possible range for eigenvalues: [{check_eigs_range(A)}]")
     print("Built-in function:")
     print(f"      eigenvalues   : {expected_eigs}")
     print(f"      eigenvectors  :\n{expected_vects}")
        

In [ ]:
A = np.array([[ -0.81417, -0.01937,  0.41372],
              [ -0.01937,  6.586211, 0.54414],
              [  0.41372,  0.00590, -0.81445]])

experiment(A)

In [ ]:
rank = 5
A = create_hilbert_matrix(rank)

experiment(A)

In [ ]:
rank = 7
A = create_hilbert_matrix(rank)

experiment(A)

In [ ]:
rank = 9
A = create_hilbert_matrix(rank)

experiment(A)